In [13]:
import numpy as np
import plotly.graph_objects as go 

# 1. Red de unidades de umbral lineal

Programa y evalúa una red de neuronas con funciones de activación escalón unitario que aproxime
la operación XNOR ($\otimes$) dada por

| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |1|
|1 |0 |1|
|1 |1 |0|



## Unidad de Umbral Lineal

Como primer paso se define la función de activación escalon, tal como se observa a continuación

$
\phi(x) = \begin{cases} 1, & \text{si } x \geq 0\\0, & \text{en caso contrario}\end{cases}
$

Es decir, si el potencial de membrana supera un valor umbral  (umbral de disparo), entonces la neurona se activa, si no lo supera, la neurona no se activa

In [2]:
def escalonUni(z):
    if z >= 0.0:
        return 1.0
    else:
        return 0.0

vEsc=np.vectorize(escalonUni)

Podemos considerar la operación XOR como una tarea de clasificación binaria a partir de 2 entradas. Por lo tanto, usaremos la función de pérdida de entropía cruzada binaria:

$$
ECB(\mathbf{y}, \mathbf{\hat{y}})  = -\sum_{i=1}^N \left[ y^{(i)} \log \hat{y}^{(i)} + (1 - y^{(i)}) \log (1 - \hat{y}^{(i)}) \right]
$$

In [3]:
def ECB(y, p):
    p[p == 0] = np.nextafter(0., 1.)
    p[p == 1] = np.nextafter(1., 0.)
    return -(np.log(p[y == 1]).sum() + np.log(1 - p[y == 0]).sum())

Para medir el rendimiento del modelo aprendido por la red neuronal densa es necesario calcular la exactitud, misma que se encuentra dada por:

$$
exactitud = \frac{correctos}{total}
$$

In [4]:
def exactitud(y, y_predicha):
    return (y == y_predicha).mean() * 100

Calculo de la función que propaga hacia adelante:
La red está compuesta de 2 capas densas (1 oculta y 1 de salida), tenemos 2 matrices de pesos
con sus correspondientes vectores de sesgos $\{\mathbf{W}^{\{1\}}, \mathbf{b}^{\{1\}}\}$ y $\{\mathbf{W}^{\{2\}}, \mathbf{b}^{\{2\}}\}$ de la capa oculta y la capa de salida respectivamente. 
Así, podemos llevar a cabo la propagación hacia adelante en esta red de la siguiente manera:

$$
	\begin{split}
				\mathbf{a}^{\{1\}} & =  \mathbf{x}^{(i)} \\
				\mathbf{z}^{\{2\}} & =  \mathbf{W}^{\{1\}} \cdot \mathbf{a}^{\{1\}} + \mathbf{b}^{\{1\}}\\
				\mathbf{a}^{\{2\}} & =  \sigma(\mathbf{z}^{\{2\}}) \\
				\mathbf{z}^{\{3\}} & =  \mathbf{W}^{\{2\}} \cdot \mathbf{a}^{\{2\}}  + \mathbf{b}^{\{2\}}\\
				\mathbf{a}^{\{3\}} & =  \sigma(\mathbf{z}^{\{3\}})\\
				\hat{y}^{(i)} & =  \mathbf{a}^{\{3\}}
			\end{split}
      $$

In [5]:
def hacia_adelante(x, W1, b1, W2, b2):
    z2 = np.dot(W1.T, x[:, np.newaxis]) + b1
    a2 = vEsc(z2)
    z3 = np.dot(W2.T, a2) + b2
    y_hat = vEsc(z3)
    return z2, a2, z3, y_hat

Definimos la función para entrenar nuestra red neuronal usando gradiente descendente. 
El descenso de gradiente es un algoritmo de optimización que permite minimizar una función haciendo actualizaciones de sus parámetros en la dirección del valor negativo de su gradiente. Aplicado a las redes neuronales, el descenso de gradiente permite ir actualizando los pesos y sesgos del modelo para reducir su error.
Para calcular el gradiente de la función de pérdida respecto a los pesos y sesgos en cada capa empleamos el algoritmo de retropropagación.


In [6]:
def retropropagacion(X, y, alpha = 0.01, n_epocas = 100, n_ocultas = 10):
    n_ejemplos = X.shape[0]
    n_entradas = X.shape[1]
        
    # Inicializamos las matrices de pesos W y V
    W1 = np.sqrt(1.0 / n_entradas) * np.random.randn(n_entradas, n_ocultas)
    b1 = np.zeros((n_ocultas, 1))
    
    W2 = np.sqrt(1.0 / n_ocultas) * np.random.randn(n_ocultas, 1)
    b2 = np.zeros((1, 1))
    
    perdidas = np.zeros((n_epocas))
    exactitudes = np.zeros((n_epocas))
    y_predicha = np.zeros((y.shape))
    for i in range(n_epocas):
        for j in range(n_ejemplos):
            z2, a2, z3, y_hat = hacia_adelante(X[j], W1, b1, W2, b2)

            # cálculo de gradientes para W2 y b2 por retropropagación
            dz3 = y_hat - y[j]
            dW2 = np.outer(a2, dz3)
            db2 = dz3

            # cálculo de gradientes para W1 y b1 por retropropagación
            dz2 = np.dot(W2, dz3) 
            dW1 = np.outer(X[j], dz2)
            db1 = dz2
            
            ####################################
            # Realizando la actualización de los parámetros
            # de forma simultánea
            W2 = W2 - alpha * dW2
            b2 = b2 - alpha * db2
            W1 = W1 - alpha * dW1
            b1 = b1 - alpha * db1

            y_predicha[j] = y_hat
            
        # Se calcula la pérdida en la época
        perdidas[i] = ECB(y, y_predicha)
        exactitudes[i] = exactitud(y, np.round(y_predicha))
        print('Epoch {0}: Pérdida = {1} Exactitud = {2}'.format(i, 
                                                              perdidas[i], 
                                                              exactitudes[i]))

    return W1, W2, perdidas, exactitudes

Aplicando la red a la operación XOR se tiene:

In [7]:
# ejemplo (XOR)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0, 1, 1, 0]]).T

Finalmente, entrenamos nuestra red con estos ejemplos por 50 épocas usando una tasa de aprendizaje $\alpha = 1.0$.

In [11]:
np.random.seed(0)
W1, W2, perdidas, exactitudes = retropropagacion(X, 
                                                 y, 
                                                 alpha = .1, 
                                                 n_epocas = 50,
                                                 n_ocultas = 2)

Epoch 0: Pérdida = 73.4736011393542 Exactitud = 50.0
Epoch 1: Pérdida = 36.7368005696771 Exactitud = 75.0
Epoch 2: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 3: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 4: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 5: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 6: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 7: Pérdida = 1525.6169444124396 Exactitud = 25.0
Epoch 8: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 9: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 10: Pérdida = 1525.6169444124396 Exactitud = 25.0
Epoch 11: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 12: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 13: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 14: Pérdida = 1525.6169444124396 Exactitud = 25.0
Epoch 15: Pérdida = 781.1768724910584 Exactitud = 50.0
Epoch 16: Pérdida = 1525.6169444124396 Exactitud = 25.0
Epoch 17: Pérdida = 1562.3537449821167 Exactitud = 0.0
Epoch 18: Pérdida 

In [17]:
fig = go.Figure(data= go.Scatter(x=list(range(len(perdidas))), y=perdidas, mode="lines+markers"))
fig.show()